In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import math

In [2]:
df = pd.read_csv('data-logistic.csv', sep=',', header=None)
val_X = df.values[:, 1:]
val_y = df.values[:, 0]

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов. 

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве {-1, 1}. Для настройки логистической регрессии мы будем решать следующую задачу:

Здесь xi1 и xi2 — значение первого и второго признаков соответственно на объекте xi. В этом задании мы будем рассматривать алгоритмы без свободного члена, чтобы упростить работу.

Градиентный шаг для весов будет заключаться в одновременном обновлении весов w1 и w2 по следующим формулам (проверьте сами, что здесь действительно выписана производная нашего функционала):



Здесь k — размер шага. 

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C в формулах, указанных выше.
Реализация в Scikit-Learn

В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск. 

В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В Scikit-Learn метрика AUC реализована функцией sklearn.metrics.roc_auc_score. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.
Материалы

    Подробнее о логистической регрессии и предсказании вероятностей с ее помощью

    Подробнее о градиентах и градиентном спуске

Инструкция по выполнению

    Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

    Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

    Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

    Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

    Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 

    Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

    Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.421. При необходимости округляйте дробную часть до трех знаков.

In [3]:
#roc_auc_score?

In [4]:
C0 = 0           #Коэффициент регуляризации (для случая БЕЗ L2-регуляризации)
C1 = 10          #Коэффициент регуляризации (для случая L2-регуляризации)
k = 0.1          #Размер шага
w0 = [0, 0]      #Начальный вектор
maxiter = 10000  #Максимальное число итераций
epsilon = 1e-5   #Критерий сходимости - евклидово расстояние между векторами весов на соседних итерациях

In [5]:
def sigmoid(x):
    return 1.0 / (1 + math.exp(-x))

In [6]:
def distance(a, b):
    return np.sqrt(np.square(a[0]-b[0]) + np.square(a[1]-b[1]))

In [7]:
def grad_desc(X, y, k, w, C, epsilon, max_iter):
    w1, w2 = w
    for i in range(max_iter):
        w1new = w1 + \
            k * np.mean(y * X[:,0] * \
                        (1. - 1./(1 + np.exp(-y * (w1 * X[:,0] + w2 * X[:,1])))) \
                        ) - k * C * w1
        
        w2new = w2 + \
            k * np.mean(y * X[:,1] * \
                        (1. - 1./(1 + np.exp(-y * (w1 * X[:,0] + w2 * X[:,1])))) \
                        ) - k * C * w2
        
        if distance((w1new, w2new), (w1, w2)) < epsilon:
            return w1new, w2new, i

        w1, w2 = w1new, w2new
    return w1new, w2new, i

In [8]:
w1_0, w2_0, i_0 = grad_desc(val_X, val_y, k, w0, C0, epsilon, maxiter)
w1_10, w2_10, i_10 = grad_desc(val_X, val_y, k, w0, C1, epsilon, maxiter)
print(w1_0, w2_0, i_0, "\n", w1_10, w2_10, i_10)

0.28781162047177644 0.09198330215925433 243 
 0.028558754546234202 0.02478013724973554 7


In [9]:
predictions_0=[]

for i in range(len(val_X)):
    s = sigmoid(w1_0 * val_X[i, 0] + w2_0 * val_X[i, 1])
    predictions_0.append(s)
answ0 = roc_auc_score(val_y, predictions_0)

In [10]:
predictions_10=[]

for i in range(len(val_X)):
    s = sigmoid(w1_10 * val_X[i, 0] + w2_10 * val_X[i, 1])
    predictions_10.append(s)
answ10 = roc_auc_score(val_y, predictions_10)

In [11]:
answer = f'{round(answ0, 3)} {round(answ10, 3)}'
print(answer)
# with open('lab8.txt', 'w') as outfile:
#     outfile.write(answer)
print("Правильный ответ 0.927 0.936")

0.927 0.936
Правильный ответ 0.927 0.936
